In [ ]:
import tensorflow as tf
from tensorflow_hub import KerasLayer
from PIL import Image
from processors.image_processor import ImageHandler, Image_preprocessor

/home/wayne/Github/PrivacyEnc/prienv/lib/python3.13/site-packages/tensorflow_hub/__init__.py:61: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import parse_version


In [ ]:
class FaceDetector:
    def __init__(self, model_url=None, confidence_threshold=0.5):
        self.confidence_threshold = confidence_threshold

        if model_url is None:
            model_url = "https://tfhub.dev/tensorflow/blazeface/1"

        try: 
            self.model = tf.keras.Sequential([
                KerasLayer(model_url)
            ])
        except Exception as e:
            raise RuntimeError("Failed to the load the model")
        
        
        
    def detect_face(self, img_rgb):
        
        preprocessed, original_shape = Image_preprocessor.preprocess(
            img_rgb,
            target_size=(128, 128),
            normalization_range = '-1-1'
        )

        # Run model inference
        predictions = self.model(preprocessed)
        
        # Extract boxes and scores
        boxes = predictions[0].numpy()   # Shape: (1, num_boxes, 4)
        scores = predictions[1].numpy()  # Shape: (1, num_boxes)
        
        # Filter detections by confidence
        face_boxes = []
        for i in range(boxes.shape[1]):
            score = scores[0, i]
            
            if score >= self.confidence_threshold:
                # Extract normalized coordinates [ymin, xmin, ymax, xmax]
                ymin, xmin, ymax, xmax = boxes[0, i]
                
                # Convert to pixel coordinates in original image
                x1 = int(xmin * orig_w)
                y1 = int(ymin * orig_h)
                x2 = int(xmax * orig_w)
                y2 = int(ymax * orig_h)
                
                # Clamp to image bounds
                x1 = max(0, x1)
                y1 = max(0, y1)
                x2 = min(orig_w, x2)
                y2 = min(orig_h, y2)
                
                face_boxes.append({
                    'bbox': (x1, y1, x2, y2),
                    'confidence': float(score)
                })
        
        return face_boxes  


        
        